In [1]:
#Importing the required libraries
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import itertools
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
layers = keras.layers

# This code was tested with TensorFlow v1.7
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.14.0


In [3]:
# Get the data: original source is here: https://www.kaggle.com/zynicide/wine-reviews/data
URL = "https://storage.googleapis.com/sara-cloud-ml/wine_data.csv"
path = tf.keras.utils.get_file(URL.split('/')[-1], URL)

49790976/49783940 [==============================] - 2s 0us/step


In [4]:
# Convert the data to a Pandas data frame
data = pd.read_csv(path)

In [5]:
# Shuffle the data
data = data.sample(frac=1)

# Print the first 5 rows
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
116680,116680,US,"Unacceptably vegetal and moldy, not to mention...",Benito Dusi Vineyard,80,28.0,California,Paso Robles,Central Coast,Zinfandel,Piedra Creek
113621,113621,Germany,"Light and very crisp, this wine even boasts hi...",QbA,84,21.0,Pfalz,NaN,NaN,Riesling,A. Christmann
14843,14843,US,"So forward and appealing right now, it's hard ...",NaN,91,24.0,California,Napa Valley,Napa,Merlot,Flora Springs
75956,75956,US,"Made from undisclosed varieties, this is a des...",Doce Nove,84,38.0,California,Paso Robles,Central Coast,Petite Sirah,Niner
36013,36013,Spain,A tight note of spearmint defines the nose. Th...,Castell de Raimat,82,10.0,Catalonia,Costers del Segre,NaN,Cabernet Sauvignon,Raimat


In [6]:
# Do some preprocessing to limit the # of wine varities in the dataset
data = data[pd.notnull(data['country'])]
data = data[pd.notnull(data['price'])]
data = data.drop(data.columns[0], axis=1) 

variety_threshold = 500 # Anything that occurs less than this will be removed.
value_counts = data['variety'].value_counts()
to_remove = value_counts[value_counts <= variety_threshold].index
data.replace(to_remove, np.nan, inplace=True)
data = data[pd.notnull(data['variety'])]

In [7]:
# Split data into train and test
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 95646
Test size: 23912


In [8]:
# Train features
description_train = data['description'][:train_size]
variety_train = data['variety'][:train_size]

# Train labels
labels_train = data['price'][:train_size]

# Test features
description_test = data['description'][train_size:]
variety_test = data['variety'][train_size:]

# Test labels
labels_test = data['price'][train_size:]

In [9]:
# Create a tokenizer to preprocess our text descriptions
vocab_size = 12000 # This is a hyperparameter, experiment with different values for your dataset
tokenize = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False)
tokenize.fit_on_texts(description_train) # only fit on train

In [10]:
# Wide feature 1: sparse bag of words (bow) vocab_size vector 
description_bow_train = tokenize.texts_to_matrix(description_train)
description_bow_test = tokenize.texts_to_matrix(description_test)

In [11]:
# Wide feature 2: one-hot vector of variety categories

# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(variety_train)
variety_train = encoder.transform(variety_train)
variety_test = encoder.transform(variety_test)
num_classes = np.max(variety_train) + 1

# Convert labels to one hot
variety_train = keras.utils.to_categorical(variety_train, num_classes)
variety_test = keras.utils.to_categorical(variety_test, num_classes)

In [13]:
# Define our wide model with the functional API
import warnings
warnings.filterwarnings("ignore")

bow_inputs = layers.Input(shape=(vocab_size,))
variety_inputs = layers.Input(shape=(num_classes,))
merged_layer = layers.concatenate([bow_inputs, variety_inputs])
merged_layer = layers.Dense(256, activation='relu')(merged_layer)
predictions = layers.Dense(1)(merged_layer)
wide_model = keras.Model(inputs=[bow_inputs, variety_inputs], outputs=predictions)

In [14]:
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
print(wide_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 12000)]      0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 40)]         0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 12040)        0           input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 256)          3082496     concatenate_1[0][0]        

In [15]:
# Deep model feature: word embeddings of wine descriptions
train_embed = tokenize.texts_to_sequences(description_train)
test_embed = tokenize.texts_to_sequences(description_test)

max_seq_length = 170
train_embed = keras.preprocessing.sequence.pad_sequences(
    train_embed, maxlen=max_seq_length, padding="post")
test_embed = keras.preprocessing.sequence.pad_sequences(
    test_embed, maxlen=max_seq_length, padding="post")

In [17]:
# Define our deep model with the Functional API
import warnings
warnings.filterwarnings("ignore")

deep_inputs = layers.Input(shape=(max_seq_length,))
embedding = layers.Embedding(vocab_size, 8, input_length=max_seq_length)(deep_inputs)
embedding = layers.Flatten()(embedding)
embed_out = layers.Dense(1)(embedding)
deep_model = keras.Model(inputs=deep_inputs, outputs=embed_out)
print(deep_model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 170)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 170, 8)            96000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1360)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1361      
Total params: 97,361
Trainable params: 97,361
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
#Compiling the model created
deep_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

In [19]:
#Combine wide and deep into one model
merged_out = layers.concatenate([wide_model.output, deep_model.output])
merged_out = layers.Dense(1)(merged_out)
combined_model = keras.Model(wide_model.input + [deep_model.input], merged_out)
print(combined_model.summary())

combined_model.compile(loss='mse',
                       optimizer='adam',
                       metrics=['accuracy'])

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 12000)]      0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 40)]         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 170)]        0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 12040)        0           input_3[0][0]                    
                                                                 input_4[0][0]              

In [20]:
# Run training
combined_model.fit([description_bow_train, variety_train] + [train_embed], labels_train, epochs=10, batch_size=128)

Epoch 1/10
95646/95646 [==============================] - 50s 520us/sample - loss: 1142.9461 - acc: 0.0000e+00
Epoch 2/10
95646/95646 [==============================] - 40s 422us/sample - loss: 888.0343 - acc: 0.0000e+00
Epoch 3/10
95646/95646 [==============================] - 38s 402us/sample - loss: 753.2520 - acc: 0.0000e+00
Epoch 4/10
95646/95646 [==============================] - 39s 408us/sample - loss: 627.4694 - acc: 0.0000e+00
Epoch 5/10
95646/95646 [==============================] - 39s 410us/sample - loss: 507.8009 - acc: 0.0000e+00
Epoch 6/10
95646/95646 [==============================] - 40s 418us/sample - loss: 395.6194 - acc: 0.0000e+00
Epoch 7/10
95646/95646 [==============================] - 40s 413us/sample - loss: 301.7766 - acc: 0.0000e+00
Epoch 8/10
95646/95646 [==============================] - 38s 401us/sample - loss: 225.5846 - acc: 0.0000e+00 - loss: 229
Epoch 9/10
95646/95646 [==============================] - 40s 415us/sample - loss: 169.4005 - acc: 0.0000e+

In [21]:
combined_model.evaluate([description_bow_test, variety_test] + [test_embed], labels_test, batch_size=128)

23912/23912 [==============================] - 10s 434us/sample - loss: 710.8781 - acc: 0.0000e+00


[710.8780591191955, 0.0]

In [22]:
# Generate predictions
predictions = combined_model.predict([description_bow_test, variety_test] + [test_embed])

In [23]:
# Compare predictions with actual values for the first few items in our test dataset
num_predictions = 40
diff = 0

for i in range(num_predictions):
    val = predictions[i]
    print(description_test.iloc[i])
    print('Predicted: ', val[0], 'Actual: ', labels_test.iloc[i], '\n')
    diff += abs(val[0] - labels_test.iloc[i])

This is the first release of this designation. The vintage was warm and ripe, and the vineyard is hard by the Sonoma border but in Napa. It's an interesting wine. It shows the power and intensity of low-yielding vines, with explosive blackberry and cherry flavors, but maintains the silky elegance you want in Pinot.
Predicted:  50.7557 Actual:  60.0 

The deep, brooding nose blends dark purple fruit with black licorice and Chinese black pepper sauce. Sage bark and anise contribute herbal tones to the palate—likely thanks to the inclusion of 14% Merlot and 11% Cabernet Franc—and blackberries rule the fruit profile.
Predicted:  20.74551 Actual:  56.0 

Low in alcohol, high in acidity and ultra-dry, this Syrah is pretty severe in the mouth. It has a tight, citrusy tartness, with sour cherry candy and orange rind flavors. Give it a good, long decant.
Predicted:  38.36195 Actual:  40.0 

Rich, thick, heavy and sweet in oak, this Cabernet is made to appeal to fans of this ripe style. It's a f

In [24]:
# Compare the average difference between actual price and the model's predicted price
print('Average prediction difference: ', diff / num_predictions)

Average prediction difference:  12.741373300552368
